# Purpose:

* For this project, I examined data on a person's height, weight and gender. 
* I am using this data to create a predictive model to classify a person into a category based on their Body Mass Index (BMI).

### Index Types:
    - 0 - Extremely Weak 
    - 1 - Weak 
    - 2 - Normal 
    - 3 - Overweight 
    - 4 - Obesity 
    - 5 - Extreme Obesity
    
   
Index Types from the Dataset are compared to those defined by the World Health Organization (WHO)
Table 1. Nutritional status

### BMI	Nutritional status: 

**Underweight:** Below 18.5

**Normal weight:** 18.5–24.9

**Pre-obesity:** 25.0–29.9

**Obesity class I:** 30.0–34.9

**Obesity class II:** 35.0–39.9

**Obesity class III:** Above 40


Source: http://www.euro.who.int/en/health-topics/disease-prevention/nutrition/a-healthy-lifestyle/body-mass-index-bmi

### How to Calculate BMI:

![final_dataset/BMI_calc.svg](attachment:final_dataset/BMI_calc.svg)
    
For more information about BMI: https://en.wikipedia.org/wiki/Body_mass_index

In [3]:
import pandas as pd

df = pd.read_csv('final_dataset/Gender_Height_Weight.csv')
df.head()

,Gender,Height,Weight,Index
0,Male,174,96,4
1,Male,189,87,2
2,Female,185,110,4
3,Female,195,104,3
4,Male,149,61,3
